## Using SQLModel to work with database tables

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

this walks through process of

 - importing modules needed
 - creates a temporary db using .env


In [ ]:

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from ewxpwsdb.db.models import WeatherStation, Reading, StationType, APIResponse
from ewxpwsdb.db.importdata import import_station_file, read_station_table


# this notebook does not use the .env file or the get_db_url() function for the app's db, 
# instead it creates a new file to be used for a temporary, empty sqlite database, deleted in the end. 
from tempfile import NamedTemporaryFile

temp_sqlitefile = NamedTemporaryFile(suffix = '.db')
temp_db_url = f"sqlite:///{temp_sqlitefile.name}"
engine = get_engine(db_url=temp_db_url)


## create new temp database to work with

In [ ]:

init_db(engine)

In [ ]:
station_file = '../data/test_stations.tsv'
import_station_file(station_file, engine = engine)


In [ ]:
from sqlmodel import select
with Session(engine) as session:
        statement = select(WeatherStation)
        results = session.exec(statement)
        stations = results.all()

len(stations)

In [ ]:
# attempt to add a duplicate
from sqlalchemy import exc
# from sqlalchemy import IntegrityError
station_list = read_station_table(station_file)
dup_ws = WeatherStation.model_validate(station_list[2])
with Session(engine) as session:
    try:
        session.add(dup_ws)
        session.commit()
        print("if this is printing, then dup records were able to be inserted into the db")
    except exc.IntegrityError as e:
        print("duplicate record caused integrity error, which is what we want!")

    


In [ ]:
our_table_names = ['WeatherStation', 'Reading', 'apiresponse', 'stationtype']
from sqlalchemy import inspect
inspector = inspect(engine)

db_tables = list(inspector.get_table_names())
db_tables

### Example SQL

In [ ]:
from sqlmodel import text

sql="select * from weatherstation inner join stationtype on weatherstation.station_type = stationtype.station_type where stationtype.station_type = 'ZENTRA';"
with Session(engine) as session:
    stmt = text(sql)
    results = session.exec(stmt)
    records = results.all()

len(records)


### Clean up

remove test database

In [ ]:
session.close()
engine.dispose()
temp_sqlitefile.close()